kudos hamidtarek

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)
import cudf as cu

import xgboost as xgb
print("XGBoost version:", xgb.__version__)

import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
%%time

import cudf as cu
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier, Pool
from sklearn.feature_selection import SelectFromModel
from tqdm.notebook import tqdm
import gc

In [ ]:
%%time
train_cudf = cu.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
data = train_cudf.to_pandas()
del train_cudf

In [ ]:
features = [c for c in data.columns if 'feature' in c]
for i in features:
    x = data[i].mean()     
    data[i] = data[i].fillna(x)

In [ ]:
# I have taked this cell from https://www.kaggle.com/jazivxt/the-market-is-reactive
# And https://www.kaggle.com/drcapa/jane-street-market-prediction-starter-xgb

#train = train[train['weight'] != 0]
#train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')

#X_train = train.loc[:, features]
#X_train = X_train.fillna(-999)
#y_train = train.loc[:, 'action']

In [ ]:
print(data.shape)

# prepare the data before splitting to train and test
data = data[data['weight']!=0]
data['action'] = (data['resp']>0)*1
print(data.shape)
#features = data.columns.str.contains('feature')

# with we will create the model
train = data.sample(frac = 0.75, random_state = 73)
print(train.shape)
#train = data.reset_index(drop=True) # if we keep it here it will raise an error in the valid set, cause if the change of the index

valid = data.drop(train.index) # take the data that does not exist in train
print(valid.shape)


train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

# it will help us track the model
#test  = data.tail(500).reset_index(drop=True)

# split the train set
X_train = train.loc[:,features] 
y_train = train.loc[:, 'action']

# split the test set
#X_test = test.loc[:, features]
#y_test = test.loc[:, 'action']


X_valid = valid.loc[:, features]
y_valid = valid.loc[:, 'action']

In [ ]:
%%time

selector = CatBoostClassifier(thread_count = -1, task_type = "GPU", devices = '-1', random_seed = 73).fit(Pool(X_train, y_train), verbose = 100) # without task_type='GPU' the fit time is 42min with GPU is 46s
list_of_tuples = list(zip(X_train.columns.values, selector.get_feature_importance())) 

df = pd.DataFrame(list_of_tuples).sort_values(by = [1]).reset_index(drop = True).rename(columns = {0: 'feat_labels', 1: 'feature_importances'})
df.head()

In [ ]:
df1 = df.tail(111) # features importance > 0.003
features = list(df1["feat_labels"])

X_train = X_train.loc[:, features]
#X_test = X_test.loc[:, features]
#print(train.shape)
print(X_train.shape)
#print(X_test.shape)

X_valid = X_valid.loc[:, features]
print(X_valid.shape)

In [ ]:
# The training part taked from here https://www.kaggle.com/xhlulu/ieee-fraud-xgboost-with-gpu-fit-in-40s

model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    missing=None,
    random_state=73,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

%time model.fit(X_train, y_train)

In [ ]:
for (test_df, sample_prediction_df) in tqdm(iter_test):
    
    X_test = test_df.loc[:, features]
    
    for i in features:
        x = X_test[i].mean()     
        X_test[i] = X_test[i].fillna(x)

    y_preds = model.predict(X_test)
    
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)